In [ ]:
# Install necessary libraries
!pip install ultralytics==8.2.71 opencv-python==4.10.0.84

In [ ]:
# Download pretrained model
!wget https://github.com/s1072489/smartHive-vision/raw/main/src/YOLOv10-Bee.pt

In [1]:
# Import necessary libraries
from ultralytics import YOLO
from collections import defaultdict
import numpy as np
import cv2 as cv

model = YOLO("YOLOv10-Bee.pt")
capture = cv.VideoCapture("video.avi")

track_history = defaultdict(lambda: [])

if not capture.isOpened():
    print("Could not mount video file")

while True:
    ret, frame = capture.read()
    
    if not ret:
        break
    
    results = model.track(frame, persist=True)
    
    # Visualize the results on the frame
    boxes = results[0].boxes.xywh.cpu()
    track_ids = results[0].boxes.id.int().cpu().tolist()

    annotated_frame = results[0].plot()

    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        
        track = track_history[track_id]
        track.append((float(x), float(y)))  # x, y center point
        
        if len(track) > 30:  # retain 90 tracks for 90 frames
            track.pop(0)

        # Draw the tracking lines
        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        cv.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

    # Display the annotated frame
    cv.imshow("YOLOv8 Tracking", annotated_frame)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

capture.release()
cv.destroyAllWindows()

Could not mount video file
